# This Notebook demonstrates how to use hooks in Strands

In [ ]:
from strands.hooks import BeforeInvocationEvent

# Register individual callbacks
def my_callback(event: BeforeInvocationEvent) -> None:
    print("Before Invocation Custom callback triggered")

In [ ]:
from strands import Agent
from strands.models.ollama import OllamaModel

# Create an Ollama model instance
ollama_model = OllamaModel(
    host="http://localhost:11434",  # Ollama server address
    model_id="qwen3-vl:4b"               # Specify which model to use
)

# Create an agent using the Ollama model
agent = Agent(model=ollama_model, hooks=[my_callback])


In [4]:
from strands.hooks import AfterInvocationEvent

def after_invocation_event_handler(event: AfterInvocationEvent) -> None:
    print("After Invocation Custom callback triggered")
    
# another way to add hook to agent, after the fact
agent.hooks.add_callback(AfterInvocationEvent, after_invocation_event_handler)

In [5]:
# invoke the agent to check the handler
agent("how many 'r' are there in 'garlic'?")

Custom callback triggered
The word "garlic" is spelled **g-a-r-l-i-c**. Let's count the occurrences of the letter 'r':

- Position 1: 'g' (not 'r')
- Position 2: 'a' (not 'r')
- Position 3: 'r' (this is one 'r')
- Position 4: 'l' (not 'r')
- Position 5: 'i' (not 'r')
- Position 6: 'c' (not 'r')

There is only **one 'r'** in the word "garlic". This is based on the standard English spelling of the word, which has six letters. The count is for the written text, not the pronunciation (e.g., "garlic" is pronounced with an 'r' sound, but there is only one 'r' in the spelling).

If you have any other questions, feel free to ask!After Invocation Custom callback triggered


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The word "garlic" is spelled **g-a-r-l-i-c**. Let\'s count the occurrences of the letter \'r\':\n\n- Position 1: \'g\' (not \'r\')\n- Position 2: \'a\' (not \'r\')\n- Position 3: \'r\' (this is one \'r\')\n- Position 4: \'l\' (not \'r\')\n- Position 5: \'i\' (not \'r\')\n- Position 6: \'c\' (not \'r\')\n\nThere is only **one \'r\'** in the word "garlic". This is based on the standard English spelling of the word, which has six letters. The count is for the written text, not the pronunciation (e.g., "garlic" is pronounced with an \'r\' sound, but there is only one \'r\' in the spelling).\n\nIf you have any other questions, feel free to ask!'}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[37.64462900161743], traces=[<strands.telemetry.metrics.Trace object at 0x11a854ad0>], accumulated_usage={'inputTokens': 2310, 'outputTokens': 22, 'totalTokens': 2332}, accumulated_metri

In [12]:
from pprint import pprint
from strands.hooks import HookProvider
from strands.hooks import HookRegistry, BaseHookEvent
from strands.hooks import AfterModelCallEvent, AfterToolCallEvent, AgentInitializedEvent, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
# all the hooks defined in the class
class LoggingHookProvider(HookProvider):
    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeInvocationEvent, self.log_event)
        registry.add_callback(AfterInvocationEvent, self.log_event)
        registry.add_callback(BeforeModelCallEvent, self.log_event)
        registry.add_callback(AfterModelCallEvent, self.log_event)
        registry.add_callback(BeforeToolCallEvent, self.log_event)
        registry.add_callback(AfterToolCallEvent, self.log_event)
        registry.add_callback(AgentInitializedEvent, self.log_event)
        registry.add_callback(MessageAddedEvent, self.log_event)
        
    def log_event(self, event: BaseHookEvent) -> None:
        print(f"==================================")
        print(f"Class: {event.__class__.__name__}")
        pprint(vars(event))
        print(f"==================================")


In [13]:
agent = Agent(model=ollama_model, hooks=[LoggingHookProvider()])

agent("what is 2 + 2?")

Class: AgentInitializedEvent
{'_disallow_writes': True,
 'agent': <strands.agent.agent.Agent object at 0x11acf3890>}
Class: BeforeInvocationEvent
{'_disallow_writes': True,
 'agent': <strands.agent.agent.Agent object at 0x11acf3890>}
Class: MessageAddedEvent
{'_disallow_writes': True,
 'agent': <strands.agent.agent.Agent object at 0x11acf3890>,
 'message': {'content': [{'text': 'what is 2 + 2?'}], 'role': 'user'}}
Class: BeforeModelCallEvent
{'_disallow_writes': True,
 'agent': <strands.agent.agent.Agent object at 0x11acf3890>}
The answer to **2 + 2** is **4**.  

This is a fundamental operation in basic arithmetic. Here's a simple breakdown:  
- **2** (two) + **2** (two) = **4** (four).  

### Why?  
- In the **decimal (base-10)** system, which is the standard used in everyday life, **2** represents the number two. When you add two groups of two, you get a total of four.  
- For example:  
  - If you have 2 apples and add another 2 apples, you now have **4 apples**.  
  - Similarly, i

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The answer to **2 + 2** is **4**.  \n\nThis is a fundamental operation in basic arithmetic. Here\'s a simple breakdown:  \n- **2** (two) + **2** (two) = **4** (four).  \n\n### Why?  \n- In the **decimal (base-10)** system, which is the standard used in everyday life, **2** represents the number two. When you add two groups of two, you get a total of four.  \n- For example:  \n  - If you have 2 apples and add another 2 apples, you now have **4 apples**.  \n  - Similarly, in mathematics, this is verified by the **commutative property of addition** (order doesn’t matter):  \n    $2 + 2 = 2 + 2 = 4$.\n\n### Edge Cases (for curiosity):  \n- **In binary (base-2)**:  \n  $2_{10} = 10_2$ (since 2 in base-10 is written as "10" in binary).  \n  Adding $10_2 + 10_2 = 100_2$, which equals **4** in base-10.  \n- **In other contexts** (e.g., modular arithmetic, non-standard number systems):  \n  The result may di